In [1]:
from matplotlib import pyplot as plt
from QDSim.sim import Simulation
from QDSim.dispatcher import repeat_execution, star_execution
from QDSim.noise import thermal_state
from QDSim.physics import root_iSWAP
from QDSim.plot import load_results
from QDSim.solvers import do_qt_mesolve

import qutip as qt
import numpy as np

# cav = thermal_state(5e9, 20e-3)
cav = qt.basis(5,0)
state = qt.tensor(cav, qt.basis(2, 0), qt.basis(2, 1))

def solve_for_delta(delta):
    target = root_iSWAP * qt.tensor(qt.basis(2,0), qt.basis(2,1))
    sim = Simulation(do_qt_mesolve, state=state, fname='time_dependent.yaml')
    sim.w_1 = sim.w_c - delta
    sim.w_2 = sim.w_c - delta
    sim._load_H()
    print('start solver for delta ' + str(delta))
    states = sim.run_solver(nsteps=1000, steps=5000, tau=1e-9)
    print('done solver')
    print('do fids ' + str(delta))
    fids = [qt.fidelity(target, s.ptrace([1, 2])) for s in states]
    print('return')
    return fids


In [2]:
detunings = [1e7,5e7,1e8]
kwarg_list = [{}]*len(detunings)
results = [solve_for_delta(d) for d in detunings]


start solver for delta 10000000.0
done solver
do fids 10000000.0
return
start solver for delta 50000000.0
done solver
do fids 50000000.0
return
start solver for delta 100000000.0
done solver
do fids 100000000.0
return


In [ ]:
times = 1e-9*np.arange(5000)
labels = ['10MHz','50MHz','100MHz']
markers = ['r-','g-','b-']#'r--','g--']#,'b--','r:']
fig = plt.figure()
ax = fig.add_subplot(111,title='iSWAP Fidelity with increasing Detuning')
for i, res in enumerate(results):
    plt.plot(times, res, markers[i], label=labels[i])
ax.legend(loc='upper right')
ax.set_xlabel('Time /s')
ax.set_ylabel('Fidelity')
plt.show()